In [1]:
import ee
import geemap
import os

In [2]:
ee.Authenticate()


Successfully saved authorization token.


In [3]:
Map = geemap.Map()

In [4]:
# Load the Nighttime Image Collection
RTMA_Image_Collection = 'NOAA/NWS/RTMA'
all_collection = ee.ImageCollection(RTMA_Image_Collection)

In [5]:
from datetime import datetime, timedelta
date_today = datetime.today().strftime('%Y-%m-%d')
date_yesterday = datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')

print("Date Today :",date_today,"| Date Yesterday :",date_yesterday)

Date Today : 2021-10-16 | Date Yesterday : 2021-10-15


In [6]:
dataset = all_collection.filter(ee.Filter.date(date_yesterday, date_today))

In [7]:
Map.add_basemap('Google Satellite') # 'Google Satellite' / Terrain

In [8]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [9]:
# Draw any shapes on the map using the Drawing tools before executing this code block
try :
    feature = Map.draw_last_feature
except:
    feature=None
if feature is None:
    """
    geom = ee.Geometry.Polygon([[[-115.413031, 35.889467],
    [-115.413031, 36.543157],
    [-114.034328, 36.543157],
    [-114.034328, 35.889467],
    [-115.413031, 35.889467]]])
    """
    geom = ee.Geometry.Polygon([[[-124.86316,56.97715],
    [-70.20644,53.78275],
    [-73.90380, 26.83366],
    [-134.74959, 27.33460]]])
    feature = ee.Feature(geom, {})

roi = feature.geometry()

from copy import deepcopy
aoi = deepcopy(roi)

In [10]:
type(roi)

ee.geometry.Geometry

In [11]:
feat = ee.Feature(roi)
Map.addLayer(feat);
Map.centerObject(feature,4);

In [12]:
Map

Map(bottom=1797.0, center=[43.4964195182664, -101.70865951901703], controls=(WidgetControl(options=['position'…

In [13]:
# sf = ee.Geometry.Point([-122.47555371521855, 37.76884708376152]);
# Amazon Rainforests
P1 = (-3.03014,-62.60020)
P2 = (-3.71241,-61.74739)

# North America
P1 = (-40,-20)
P2= (40,20)

# South America
P1 = (7.3625,266.9549)
P2= (-52.9089,335.1825)

x1,y1 = P1
x2,y2 = P2

# roi = ee.Geometry.Rectangle([-40, -20, 40, 20])
roi = ee.Geometry.Rectangle([x1,y1,x2,y2])
bounded = dataset.filterBounds(roi)

In [14]:
def getMidpoint(P1,P2):
  x1,y1 = P1
  x2,y2 = P2

  xCenter = (x1 + x2) / 2
  yCenter = (y1 + y2) / 2

  return xCenter,yCenter

PCenter = getMidpoint(P1,P2)

xCenter,yCenter = PCenter

In [15]:
# Extract the first image
first_image = bounded.first()
 
# Display band information about the image
first_image.bandNames().getInfo()

['HGT',
 'PRES',
 'TMP',
 'DPT',
 'UGRD',
 'VGRD',
 'SPFH',
 'WDIR',
 'WIND',
 'GUST',
 'VIS',
 'TCDC']

In [16]:
lengthCollection = bounded.size()
listOfImages = ee.Image(bounded.toList(bounded.size()));
firstImage = ee.Image(listOfImages.get(0));
secondImage = ee.Image(listOfImages.get(1));


In [17]:
lastImage = ee.Image(listOfImages.get(lengthCollection.subtract(1)));

In [18]:
# Compute the median in each band, each pixel.
# Band names are B1_median, B2_median, etc.
median = bounded.reduce(ee.Reducer.median());
median.bandNames().getInfo()

['HGT_median',
 'PRES_median',
 'TMP_median',
 'DPT_median',
 'UGRD_median',
 'VGRD_median',
 'SPFH_median',
 'WDIR_median',
 'WIND_median',
 'GUST_median',
 'VIS_median',
 'TCDC_median']

In [19]:
# The output is an Image.  Add it to the map.
vis_param = {"bands": ['TMP_median', 'WDIR_median', 'WIND_median'], "gamma": 0.8};

In [20]:
# Map.setCenter(-122.3355, 37.7924, 9);

Map.addLayer(median, vis_param, RTMA_Image_Collection, True, 1)

In [21]:
Map

Map(bottom=1803.0, center=[43.13306116240615, -99.84375], controls=(WidgetControl(options=['position'], widget…

In [23]:
type(Map)

geemap.geemap.Map

In [24]:
rgb_img = geemap.ee_to_numpy(median, region=aoi)
print(rgb_img.shape)

Image.sampleRectangle: Fully masked pixels / pixels outside of the image footprint when sampling band 'HGT_median' with no default value set. Note that calling sampleRectangle() on an image after ee.Image.clip() may result in a sampling bounding box outside the geometry passed to clip().


AttributeError: 'NoneType' object has no attribute 'shape'